<a href="https://colab.research.google.com/github/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021/blob/main/examples/A2C_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git to /tmp/pip-req-build-7vjrrevq
  Running command git clone -q https://github.com/leoxiang66/CS-285-at-UC-Berkeley-Deep-Reinforcement-Learning-2021.git /tmp/pip-req-build-7vjrrevq
  Created wheel for RLalgos: filename=RLalgos-0.1.3-py3-none-any.whl size=8155 sha256=2a8d34d7a6e627756961d7018c805271f962e2fc49266bfe8e4846eedf4a8175
  Stored in directory: /tmp/pip-ephem-wheel-cache-osy8fsrz/wheels/4c/56/f2/8ddd79049992cb6f5396813f1f0ab97eac7ab2633c7c9eb79d
Successfully built RLalgos


In [2]:
from RLalgos import A2C as A2CAgent

In [3]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
import numpy as np

In [4]:
#@title Config A2C {display-mode: "form", run: "auto"}
from pprint import pprint
env_id = 'CartPole-v1'  #@param ["CartPole-v1", "Acrobot-v1", "MountainCar-v0"]
value_learning_rate = 0.001  #@param {type: "number"}
actor_learning_rate = 0.001  #@param {type: "number"}
gamma = 1  #@param {type: "number"}
entropy = 1  #@param {type: "number"}
seed = 1  #@param {type: "integer"}
#@markdown ---

config_a2c = {
    'env_id': env_id,
    'gamma': gamma,
    'seed': seed,
    'value_network': {'learning_rate': value_learning_rate},
    'actor_network': {'learning_rate': actor_learning_rate},
    'entropy': entropy
}

print("Current config_a2c is:")
pprint(config_a2c)

Current config_a2c is:
{'actor_network': {'learning_rate': 0.001},
 'entropy': 1,
 'env_id': 'CartPole-v1',
 'gamma': 1,
 'seed': 1,
 'value_network': {'learning_rate': 0.001}}


In [5]:
agent = A2CAgent(config_a2c)
rewards = agent.training_batch(1000, 256)

/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym

Epoch 0/1000: Mean rewards: 22.64, Std: 11.85
Epoch 50/1000: Mean rewards: 37.08, Std: 20.54
Epoch 100/1000: Mean rewards: 64.68, Std: 41.18
Epoch 150/1000: Mean rewards: 82.0, Std: 44.5
Epoch 200/1000: Mean rewards: 182.94, Std: 87.27
Epoch 250/1000: Mean rewards: 218.14, Std: 85.63
Epoch 300/1000: Mean rewards: 291.76, Std: 127.34
Epoch 350/1000: Mean rewards: 348.24, Std: 133.85
Epoch 400/1000: Mean rewards: 404.06, Std: 119.66
Epoch 450/1000: Mean rewards: 376.48, Std: 102.97
Epoch 500/1000: Mean rewards: 463.82, Std: 78.56
Epoch 550/1000: Mean rewards: 409.14, Std: 96.66
Epoch 600/1000: Mean rewards: 474.84, Std: 56.25
Epoch 650/1000: Mean rewards: 457.72, Std: 87.65
Epoch 700/1000: Mean rewards: 429.46, Std: 99.18
Epoch 750/1000: Mean rewards: 489.18, Std: 47.07
Epoch 800/1000: Mean rewards: 428.58, Std: 90.44
Epoch 850/1000: Mean rewards: 478.78, Std: 69.87
Epoch 900/1000: Mean rewards: 476.58, Std: 78.6
Epoch 950/1000: Mean rewards: 476.48, Std: 75.73
Epoch 999/1000: Mean rewar

In [6]:
agent.evaluate()

500.0